<a href="https://colab.research.google.com/github/Reennon/multigec-models/blob/main/notebooks/aya_expanse_8b/multigec/multigec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from google.colab import userdata

os.environ["GIT_TOKEN"] = userdata.get('git_token')

In [2]:
!git clone https://$GIT_TOKEN@github.com/Reennon/multigec-models.git

Cloning into 'multigec-models'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 59 (delta 13), reused 39 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 92.20 KiB | 11.53 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [3]:
%cd multigec-models

/content/multigec-models


In [28]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 324 bytes | 324.00 KiB/s, done.
From https://github.com/Reennon/multigec-models
   ed7eb51..e447acc  main       -> origin/main
Updating ed7eb51..e447acc
Fast-forward
 params/aya_expanse_8b.yaml | 1 -
 1 file changed, 1 deletion(-)


In [5]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit -q
!pip install flash-attn --no-build-isolation -q
!pip install wtpsplit==2.1.1 -q
!pip install syntok==1.4.4 -q
!pip install omegaconf -q
!pip install wandb -q
!pip install --upgrade transformers trl -q
!pip install pandas numpy -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [29]:
import os

from omegaconf import OmegaConf
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from src.utils.multigec import sentences, LANG_TO_CODE, LANG_CODE_TO_TOKEN
from langchain_core.prompts import PromptTemplate

from src.instruction_templates import multigec_prompts

import torch
import wandb

from transformers import BitsAndBytesConfig
from tqdm import tqdm
from trl.trainer import ConstantLengthDataset
import pandas as pd
from datasets import Dataset
from transformers.trainer_callback import EarlyStoppingCallback

from transformers import TrainingArguments
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig

tqdm.pandas()

In [30]:
parameters = OmegaConf.load("./params/aya_expanse_8b.yaml")

In [31]:
track     = "minimal"
model_name = "aya-expanse-8b"
fine_tuned_model_name = f"aya-expanse-8b-multigec"
experiment_name = f"multigec-{track}-{model_name}"

hf_key   = userdata.get("hf_key")
secret_wandb = userdata.get("wandb_key")
in_path  = f"/gdrive/MyDrive/datasets/multigec/multigec_{track}.csv"

# Path where the output will be saved to
out_path = f"/gdrive/MyDrive/preds/multigec/multigec_test_{track}.csv"
out_model_dir = f"/gdrive/MyDrive/models/multigec/{fine_tuned_model_name}"
QUANTIZE_4BIT = True
device   = "cuda:0"

In [13]:
wandb_project_name = f'{model_name.upper()}-multigec-{track}'

wandb.login(key = secret_wandb)

/usr/local/lib/python3.11/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rkovalch (rkovalchuk) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [14]:
login(hf_key)

In [15]:
import gc

def clear_gpu_memory():
    torch.cuda.empty_cache()
    print(gc.collect())
clear_gpu_memory()

205


In [16]:
!env TORCH_USE_CUDA_DSA=1

SHELL=/bin/bash
NV_LIBCUBLAS_VERSION=12.5.3.2-1
NVIDIA_VISIBLE_DEVICES=all
COLAB_JUPYTER_TRANSPORT=ipc
NV_NVML_DEV_VERSION=12.5.82-1
NV_CUDNN_PACKAGE_NAME=libcudnn9-cuda-12
CGROUP_MEMORY_EVENTS=/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events
NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.22.3-1+cuda12.5
NV_LIBNCCL_DEV_PACKAGE_VERSION=2.22.3-1
VM_GCE_METADATA_HOST=169.254.169.253
HOSTNAME=c8a0d8a8118f
LANGUAGE=en_US
TBE_RUNTIME_ADDR=172.28.0.1:8011
COLAB_TPU_1VM=
GCE_METADATA_TIMEOUT=3
NVIDIA_REQUIRE_CUDA=cuda>=12.5 brand=unknown,driver>=470,driver<471 brand=grid,driver>=470,driver<471 brand=tesla,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=vapps,driver>=470,driver<471 brand=vpc,driver>=470,driver<471 brand=vcs,driver>=470,driver<471 brand=vws,driver>=470,driver<471 brand=cloudgaming,driver>=470,driver<471 brand=unknown,driver>=535,dr

In [17]:
checkpoint = "CohereForAI/aya-expanse-8b"
quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
config = AutoConfig.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    config=config,
    quantization_config=quantization_config,
    torch_dtype="bfloat16",
    device_map=device,
    attn_implementation="flash_attention_2",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/8.64k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [18]:
multigec_df = pd.read_csv(in_path)
train_df = multigec_df.loc[multigec_df.loc[:, "split"] == "train"]
val_df = multigec_df.loc[multigec_df.loc[:, "split"] == "val"]

In [19]:
def formatting_prompts_func(example):
    language_code = LANG_TO_CODE[example["language"]]
    language_token = LANG_CODE_TO_TOKEN[language_code]

    user_input = example['feature']
    prompt_template = multigec_prompts[example["language"]].prompt_template
    instruction = prompt_template.format(original_text=user_input)
    target = example['target']

    text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{target}"

    return text

In [20]:
num_added_toks = tokenizer.add_tokens(
    [v for v in LANG_CODE_TO_TOKEN.values()],
    special_tokens=True
)
model.resize_token_embeddings(len(tokenizer))

Embedding(255040, 4096, padding_idx=0)

In [21]:
training_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

seq_length = 1300

cld_train_dataset = ConstantLengthDataset(
    tokenizer=tokenizer,
    dataset=training_dataset,
    #dataset_text_field='feature',
    seq_length=seq_length,
    eos_token_id=tokenizer.eos_token_id,
    shuffle=False,
    append_concat_token=True,
    add_special_tokens=True,
    formatting_func=formatting_prompts_func,
)
cld_val_dataset = ConstantLengthDataset(
    tokenizer=tokenizer,
    dataset=val_dataset,
    #dataset_text_field='feature',
    seq_length=int(seq_length/2),
    eos_token_id=tokenizer.eos_token_id,
    shuffle=False,
    append_concat_token=True,
    add_special_tokens=True,
    formatting_func=formatting_prompts_func,
)

In [ ]:
# parameters.training["gradient_accumulation_steps"] = 8
# parameters.training["per_device_train_batch_size"] = 5
# parameters.training["per_device_eval_batch_size"] = 2
# # Try these
# parameters.training["lr_scheduler_type"] = "cosine"
# parameters.training["max_grad_norm"] = 1.0
# parameters.training["output_dir"] = "results"
# parameters.training["num_train_epochs"] = 12
# parameters.training["eval_strategy"] = "steps"
# parameters.training["save_strategy"] = "steps"        # Save model after every step
# parameters.training["metric_for_best_model"] = "eval_loss"
# parameters.training["greater_is_better"] = False
# parameters.training["group_by_length"] = False # we already use CLD
# parameters.training["save_total_limit"] = 3           # Keep only the 3 most recent checkpoints
# parameters.training["load_best_model_at_end"] = True # EarlyStoppingCallback requires load_best_model_at_end = True
# parameters.training["warmup_steps"] = 50
# parameters.training["learning_rate"] = 3e-5
# parameters.training["save_steps"] = 25
# parameters.training["weight_decay"] = 0.0
# early_stopping_patience = 75

In [ ]:
cld_train_dataset

In [ ]:
dict(parameters.training)

In [ ]:
# def formatting_prompts_func_old(example):
#     output_texts = []
#     for i in range(len(example['inputs'])):
#         language_code = LANG_TO_CODE[example["language"]]
#         language_token = LANG_CODE_TO_TOKEN[language_code]

#         user_input = example['inputs'][i]
#         prompt_template = multigec_prompts[example["language"]]
#         instruction = prompt_template.format(original_text=user_input)
#         target = example['targets'][i]


#         text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{target}"
#         output_texts.append(text)
#     return output_texts

# def formatting_prompts_func(example):
#     language_code = LANG_TO_CODE[example["language"]]
#     language_token = LANG_CODE_TO_TOKEN[language_code]

#     user_input = example['feature']
#     prompt_template = multigec_prompts[example["language"]]
#     instruction = prompt_template.format(original_text=user_input)
#     target = example['target']

#     text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{target}"

#     return text

In [ ]:
run = wandb.init(
    project=wandb_project_name,
    job_type="training",
    anonymous="allow"
)

wandb.config.update(dict(parameters.training))

peft_config = LoraConfig(
    r=parameters.lora.r,
    lora_alpha=parameters.lora.lora_alpha,
    target_modules=list(parameters.lora.target_modules),
    bias=parameters.lora.bias,
    task_type=parameters.lora.task_type
)
training_arguments = SFTConfig(
    **parameters.training,
    packing=True,
    max_seq_length=seq_length,
    output_dir=out_model_dir,
)
trainer = SFTTrainer(
    resume_from_checkpoint=True,
    model=model,
    train_dataset=cld_train_dataset,
    eval_dataset=cld_val_dataset,
    peft_config=peft_config,
    args=training_arguments,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=parameters.early_stopping.early_stopping_patience)],
    #formatting_func=formatting_prompts_func,
)

with torch.backends.cuda.sdp_kernel(
    enable_flash=True,
    enable_math=False,
    enable_mem_efficient=False
):
    trainer.train()

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_config.py:184: DeprecationWarning: `max_seq_length` is deprecated and will be remove in version 0.20.0. Use `max_length` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_config.py:184: DeprecationWarning: `max_seq_length` is deprecated and will be remove in version 0.20.0. Use `max_length` instead.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


In [ ]:
import time

run_name = wandb.run.name
model_out_path = f"/gdrive/MyDrive/models/multigec/{run_name}"
os.makedirs(model_out_path, exist_ok=True)
trainer.model.save_pretrained(model_out_path)
wandb.save(model_out_path + "/*")
model.config.use_cache = True
model.eval()
time.sleep(30)

In [ ]:
test_df = multigec_df.loc[multigec_df.loc[:, "split"] == "test"]

def make_correction(
    model,
    input: pd.Series,
    tokenizer,
    parameters: dict,
    seq_lenght: int
):
    language: str = input["language"]
    language_code: str = LANG_TO_CODE[language]
    language_token: str = LANG_CODE_TO_TOKEN[language_code]
    prompt_template: PromptTemplate = multigec_prompts[language]

    # Prepare inputs
    inputs: list[str] = sentences(input)
    inputs = [language_token + prompt_template.format(original_text=input) for input in inputs]
    inputs: list[dict[str, str]] = get_message_format(inputs)
    input_ids = tokenizer.apply_chat_template(
        inputs,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
    )
    input_ids = input_ids.to(model.device)
    prompt_padded_len = len(input_ids[0])

    # Generate corrections
    gen_tokens = model.generate(
        input_ids,
        temperature=parameters.baseline.temperature,
        top_p=parameters.baseline.top_p,
        top_k=parameters.baseline.top_k,
        max_new_tokens=seq_lenght,
        do_sample=True,
    )
    gen_tokens = [
        gt[prompt_padded_len:] for gt in gen_tokens
    ]
    outputs: list[str] = tokenizer.batch_decode(
        gen_tokens,
        skip_special_tokens=True
    )
    # Join divided texts if any
    correction = "".join(outputs)

    return correction

test_df.progress_apply(lambda x: make_correction(
    model=model,
    input=x,
    tokenizer=tokenizer,
    parameters=parameters,
    seq_lenght=seq_length,
))
test_df

In [ ]:
test_df.to_csv(out_path)

In [ ]:
wandb.finish()

In [ ]:
raise Exception e

In [ ]:
!nvidia-smi

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!pip install pynvml -q

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
gc.collect()

In [ ]:
import time
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

def wait_until_enough_gpu_memory(min_memory_available, max_retries=10, sleep_time=5):
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(torch.cuda.current_device())

    for _ in range(max_retries):
        clear_gpu_memory()
        info = nvmlDeviceGetMemoryInfo(handle)
        if info.free >= min_memory_available:
            break
        print(f"Waiting for {min_memory_available} bytes of free GPU memory. Retrying in {sleep_time} seconds...")
        time.sleep(sleep_time)
    else:
        raise RuntimeError(f"Failed to acquire {min_memory_available} bytes of free GPU memory after {max_retries} retries.")

# Usage example
min_memory_available = 38 * 1024 * 1024 * 1024  # 30GB
clear_gpu_memory()
wait_until_enough_gpu_memory(min_memory_available)

In [ ]:
del model

In [ ]:
del trainer